# Pangea AI Security Tools

- Defend against prompt injection.
- Prevent the leakage and exposure of sensitive information, including:
  - Personally Identifiable Information (PII)
  - Protected Health Information (PHI)
  - Financial data
  - Secrets
  - Intellectual property
  - Profanity
- Remove malicious content from input and output, such as IP addresses, domains, and URLs.
- Monitor user inputs and model responses to enable comprehensive threat analysis, auditing, and compliance.

## Prerequisites

### OpenAI API key

The examples below use OpenAI models. To run them, get your [OpenAI API key](https://platform.openai.com/api-keys) and export it as an environment variable:

- `OPENAI_API_KEY`

### Pangea project

Sign up for a free [Pangea account](https://pangea.cloud/signup) to host the security services used in these tools.

After creating your account, click **Skip** on the **Get started with a common service** screen. This will take you to the Pangea User Console, where you can enable the individual services required for the tools.

To learn more about Pangea services and their capabilities, visit the Pangea website:
- [AI Guard](https://pangea.cloud/services/ai-guard/)
- [Redact](https://pangea.cloud/services/redact/)
- [Domain Intel](https://pangea.cloud/services/domain-intel/reputation/)
- [IP Intel](https://pangea.cloud/services/ip-intel/reputation/)
- [URL Intel](https://pangea.cloud/services/url-intel/)
- [Secure Audit Log](https://pangea.cloud/services/secure-audit-log/)

## Installation

In [ ]:
%%bash
pip install langchain-community langgraph langchain-openai pangea-sdk==5.2.0b2

## Tools

Run Pangea tools using agents or invoke them as a Runnable within chains.

### AI Guard

#### Enable the AI Guard service

1. Open your [Pangea User Console](https://console.pangea.cloud).  
2. Click **AI Guard** in the left-hand sidebar and follow the prompts, accepting all defaults.  
3. When finished, click **Done** and then **Finish**. The enabled service will be marked with a green dot.
4. On the service **Overview** page, capture the **Default Token** and **Domain** values by clicking their respective tiles. Save these values in the appropriate environment variables:
    - `PANGEA_DOMAIN`
    - `PANGEA_AI_GUARD_TOKEN`

For more information on setting up the underlying service and its usage, visit the [AI Guard documentation](https://pangea.cloud/docs/ai-guard/).

#### Use AI Guard as a Runnable in chains

1. Set up the environment.

In [1]:
import os
from dotenv import load_dotenv
from pydantic import SecretStr

load_dotenv()

openai_api_key = SecretStr(os.getenv("OPENAI_API_KEY"))
pangea_domain = os.getenv("PANGEA_DOMAIN")
pangea_ai_guard_token = SecretStr(os.getenv("PANGEA_AI_GUARD_TOKEN"))

2. Define the model.

In [2]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model_name="gpt-3.5-turbo", openai_api_key=openai_api_key.get_secret_value(), temperature=0)

3. Instantiate AI Guard.

   Apply an AI Guard recipe to content sent to the LLM. For example, use the [LLM Prompt Pre-Send](https://pangea.cloud/docs/ai-guard/recipes#llm-prompt-pre-send) (`pangea_llm_prompt_guard`) recipe to prevent sensitive or high-risk information from being submitted to a public LLM, such as ChatGPT:
   - Defang malicious links (e.g., IPs, URLs, domains).
   - Redact specific personally identifiable information (PII) and secrets in the prompt according to the rules defined in the recipe.

   Customize recipes by adding, removing, or modifying rules, and discover, create, and configure additional recipes in your [Pangea User Console](https://console.pangea.cloud/service/ai-guard/recipes).

In [3]:
from langchain_community.tools.pangea.ai_guard import PangeaAIGuard, PangeaConfig

pangea_config = PangeaConfig(domain=pangea_domain)
pangea_ai_guard_tool = PangeaAIGuard(token=pangea_ai_guard_token, config=pangea_config, recipe="pangea_llm_prompt_guard")

/var/folders/4f/53ppb2rn1s14bhl_l8md18yw0000gn/T/ipykernel_7970/3964412347.py:4: LangChainBetaWarning: Pangea AI Guard service is in beta. Subject to change.
  pangea_ai_guard_tool = PangeaAIGuard(token=pangea_ai_guard_token, config=pangea_config, recipe="pangea_llm_prompt_guard")


4. Create the prompt.

In [4]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([("human", "{input}")])

query = """
Hi, I am Bond, James Bond. I am looking for a job. Please write me a very short resume.

I am skilled in international espionage, covert operations, and seduction.

Include a contact header:
Email: j.bond@mi6.co.uk
Phone: +44 20 0700 7007
Address: Universal Exports, 85 Albert Embankment, London, United Kingdom
"""

5. Use AI Guard in a chain

   In the following example, the user's initial query is processed using the AI Guard recipe. The sanitized input is then added to the prompt sent to the LLM. Depending on your use case, you can apply AI Guard at different steps in your chain by invoking it with a string.

In [7]:
from langchain_core.output_parsers import StrOutputParser

chain = (
  prompt
  | pangea_ai_guard_tool
  | model
  | StrOutputParser()
)

print(chain.invoke({"input": query}))

Resume:

Name: ****
Skills: International espionage, covert operations, seduction
Contact:
Email: <EMAIL_ADDRESS>
Phone: +44 20 0700 7007
Address: Universal Exports, 85 *****************, London, United Kingdom


Note that only some personally identifiable information has been replaced or masked by the AI Guard tool. To apply stricter redaction rules, update the service recipes in your [Pangea User Console](https://console.pangea.cloud/service/ai-guard/recipes).

The same chain, without AI Guard protection, will submit sensitive information to the LLM, providing it with personal context.

In [39]:
from langchain_core.output_parsers import StrOutputParser

chain = (
  prompt
  | model
  | StrOutputParser()
)

print(chain.invoke({"input": query}))

Resume:

Name: Bond, James Bond
Email: j.bond@mi6.co.uk
Phone: +44 20 0700 7007
Address: Universal Exports, 85 Albert Embankment, London, United Kingdom

Skills:
- International espionage
- Covert operations
- Seduction

Experience:
- MI6 agent for over 20 years
- Successfully completed numerous high-profile missions
- Expert in gathering intelligence and eliminating threats

Education:
- Graduated from MI6 Academy with top honors

References available upon request.


#### Use AI Guard as an agent tool

1. Set up the environment.

In [40]:
import os
from dotenv import load_dotenv
from pydantic import SecretStr

load_dotenv()

openai_api_key = SecretStr(os.getenv("OPENAI_API_KEY"))
pangea_domain = os.getenv("PANGEA_DOMAIN")
pangea_ai_guard_token = SecretStr(os.getenv("PANGEA_AI_GUARD_TOKEN"))

2. Define the model.

In [41]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

model = ChatOpenAI(model_name="gpt-4o-mini", openai_api_key=openai_api_key.get_secret_value(), temperature=0)

3. Instantiate AI Guard.

In [42]:
from langchain_community.tools.pangea.ai_guard import PangeaAIGuard, PangeaConfig
import sys

pangea_config = PangeaConfig(domain=pangea_domain)
pangea_ai_guard_tool = PangeaAIGuard(pangea_token=pangea_ai_guard_token, config=pangea_config, recipe="pangea_llm_response_guard")

4. Use AI Guard tool with an agent

   The example below demonstrates how the `pangea_llm_response_guard` recipe can be used to defang malicious IP addresses, domains, and URLs that may be included in an LLM response to the user. You can configure the exact defanging and redaction rules by updating the service recipes in your [Pangea User Console](https://console.pangea.cloud/service/ai-guard/recipes).

In [45]:
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool

@tool
def search_tool(data):
    """Call to perform search"""

    return """
    47.84.32.175
    37.44.238.68
    47.84.73.221
    47.236.252.254
    34.201.186.27
    52.89.173.88
    """

tools = [search_tool, pangea_ai_guard_tool]

query = """
Hi, I am Bond, James Bond. I've got a job to monitor IPs found in MI6 network traffic.
Please find me the most recent ones, you copy?
"""

system_message="Always use AI Guard before your final response to keep it safe for the user."

langgraph_agent_executor = create_react_agent(model, tools, state_modifier=system_message)

state = langgraph_agent_executor.invoke({"messages": [("human", query)]})

print(state["messages"][-1].content)

Here are the most recent IP addresses found in the MI6 network traffic:

- 47[.]84[.]32[.]175
- 37[.]44[.]238[.]68
- 47[.]84[.]73[.]221
- 47[.]236[.]252[.]254
- 34.201.186.27
- 52.89.173.88

Let me know if you need anything else, 007!


In the above example, safe IPs are mixed with those listed on the [IPsum Threat Intelligence Feed](https://github.com/stamparm/ipsum) site. The AI Guard tool defangs IP addresses it identifies as dangerous, reducing the risk of users inadvertently using them.

Because IP addresses are not inherently dangerous to expose to the user, the pre-built agent is configured via a system message to apply the service to the final result. Alternatively, you can create your own agent and implement a more deterministic approach to ensure the LLM's response is thoroughly sanitized by the service.

#### Use AI Guard as a standalone tool

You can also call the AI Guard tool directly as needed.

In [44]:
print(pangea_ai_guard_tool.run("Ping me at example@example.com"))
print(pangea_ai_guard_tool.invoke("Take my SSN: 234-56-7890"))

Ping me at <EMAIL_ADDRESS>
Take my SSN: ***********


### Domain Intel Guard

#### Enable the Domain Intel service

1. Open your [Pangea User Console](https://console.pangea.cloud).  
2. Click **Domain Intel** in the left-hand sidebar and follow the prompts, accepting all defaults.  
3. When finished, click **Done** and then **Finish**. The enabled service will be marked with a green dot.
4. On the service **Overview** page, capture the **Default Token** and **Domain** values by clicking their respective tiles. Save these values in the appropriate environment variables:
    - `PANGEA_DOMAIN`
    - `PANGEA_DOMAIN_INTEL_TOKEN`
5. Click **Reputation** in the left-hand sidebar, then select a default provider.

For more information on setting up the underlying service and its usage, visit the [Domain Intel documentation](https://pangea.cloud/docs/domain-intel/).

#### Use Domain Intel Guard as a Runnable in chains

1. Set up the environment.

In [14]:
import os
from dotenv import load_dotenv
from pydantic import SecretStr

load_dotenv()

openai_api_key = SecretStr(os.getenv("OPENAI_API_KEY"))
pangea_domain = os.getenv("PANGEA_DOMAIN")
pangea_domain_intel_token = SecretStr(os.getenv("PANGEA_DOMAIN_INTEL_TOKEN"))

2. Define the model.

In [15]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model_name="gpt-4o-mini", openai_api_key=openai_api_key.get_secret_value(), temperature=0)

3. Instantiate Domain Intel Guard.

In [19]:
from langchain_community.tools.pangea.domain_intel_guard import PangeaDomainIntelGuard, PangeaConfig

pangea_config = PangeaConfig(domain=pangea_domain)
pangea_domain_intel_guard_tool = PangeaDomainIntelGuard(token=pangea_domain_intel_token, config=pangea_config, threshold=80)

4. Create the prompt.

In [16]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompt_values import ChatPromptValue
from langchain_core.messages import SystemMessage

def rag(input: ChatPromptValue) -> ChatPromptValue:
    """
    Simulates a Retrieval-Augmented Generation (RAG) process by appending a fabricated
    IT-related email to the input chat messages. This example demonstrates how spam emails
    can introduce data poisoning, leading to an indirect prompt injection attack.
    """

    messages = input.to_messages()

    message = SystemMessage("""
    Email

    From: IT Admin
    Subject: Update VPN configuration

    Hi team, your friendly IT admin here. Use `neuzeitschmidt.site` for the VPN server address.

    Thanks, and don't forget to plug you computer in!
    """)

    messages.append(message)

    return ChatPromptValue(messages=messages)

prompt = ChatPromptTemplate.from_messages([("human", "{input}")])

query = """
Read me back the most recent email from IT.
"""

5. Use Domain Intel Guard in a chain

In [17]:
from langchain_core.output_parsers import StrOutputParser

chain = (
  prompt
  | rag
  | pangea_domain_intel_guard_tool
  | model
  | StrOutputParser()
)

print(chain.invoke({"input": query}))

I'm sorry, but I can't access your emails or any personal data. However, I can help you draft a response or provide guidance on how to check your email. Let me know what you need!


In the above example, a malicious email context was introduced to the user's question through data poisoning. However, the Domain Intel Guard tool blocked the content containing the harmful domain, rendering the LLM's response harmless.

Without the protection of the Domain Intel Guard tool, the same chain might inadvertently return the malicious domain to the user:

In [18]:
from langchain_core.output_parsers import StrOutputParser

chain = (
  prompt
  | rag
  | model
  | StrOutputParser()
)

print(chain.invoke({"input": query}))

The most recent email from IT is as follows:

**From:** IT Admin  
**Subject:** Update VPN configuration  

Hi team, your friendly IT admin here. Use `neuzeitschmidt.site` for the VPN server address.

Thanks, and don't forget to plug your computer in!


#### Use Domain Intel Guard as an agent tool

1. Set up the environment.

In [15]:
import os
from dotenv import load_dotenv
from pydantic import SecretStr

load_dotenv()

openai_api_key = SecretStr(os.getenv("OPENAI_API_KEY"))
pangea_domain = os.getenv("PANGEA_DOMAIN")
pangea_domain_intel_token = SecretStr(os.getenv("PANGEA_DOMAIN_INTEL_TOKEN"))

2. Define the model.

In [16]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model_name="gpt-4o-mini", openai_api_key=openai_api_key.get_secret_value(), temperature=0)

3. Instantiate IP Intel Guard.

In [17]:
from langchain_community.tools.pangea.domain_intel_guard import PangeaDomainIntelGuard, PangeaConfig

pangea_config = PangeaConfig(domain=pangea_domain)
pangea_domain_intel_guard_tool = PangeaDomainIntelGuard(token=pangea_domain_intel_token, config=pangea_config, threshold=80)

4. Define tools, the example context, and the agent. 

   The example below demonstrates how IP Intel Guard can prevent an agent from responding to the user with malicious information included in the content.

In [18]:
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool

@tool
def search_tool(data):
    """Call to perform email search"""

    """
    Simulates an email search tool finding a fabricated IT-related email.
    This example demonstrates how spam emails can introduce data poisoning,
    leading to an indirect prompt injection attack.
    """

    return """
    Email

    From: IT Admin
    Subject: Update VPN configuration

    Hi team, your friendly IT admin here. Use `neuzeitschmidt.site` for the VPN server address.

    Thanks, and don't forget to plug you computer in!
    """

query = """
Read me back the most recent email from IT.
"""

5. Use the Domain Intel Guard tool with the agent.

    In the following example, we instruct the pre-built agent via a system message to stop execution if any malicious content is found. Alternatively, you can create your own agent and implement a more deterministic approach to ensure the LLM's response is safe for the user.

In [19]:
tools = [search_tool, pangea_domain_intel_guard_tool]

system_message = """
Always use the Domain Intel Guard tool before your final response.
If malicious content is detected, respond with: Malicious content found in the response.
"""

langgraph_agent_executor = create_react_agent(model, tools, state_modifier=system_message)

state = langgraph_agent_executor.invoke({"messages": [("human", query)]})

print(state["messages"][-1].content)

Malicious content found in the response.


In the above example, a malicious email context was introduced to the user's question through data poisoning. However, the Domain Intel Guard tool informed the agent about the malicious content.

Without the protection of the Domain Intel Guard tool, the same agent might inadvertently return the malicious domain to the user:

In [20]:
tools = [search_tool]

langgraph_agent_executor = create_react_agent(model, tools)

state = langgraph_agent_executor.invoke({"messages": [("human", query)]})

print(state["messages"][-1].content)

The most recent email from IT is as follows:

**From:** IT Admin  
**Subject:** Update VPN configuration  

Hi team, your friendly IT admin here. Use `neuzeitschmidt.site` for the VPN server address.

Thanks, and don't forget to plug your computer in!


#### Use Domain Intel Guard as a standalone tool

You can also call the Domain Intel Guard tool directly as needed.

In [26]:
print(pangea_domain_intel_guard_tool.run("neuzeitschmidt.site"))
print(pangea_domain_intel_guard_tool.invoke("neuzeitschmidt.site"))

Malicious domains found in the provided input.
Malicious domains found in the provided input.


SEPARATOR

### IP Intel Guard

#### Enable the IP Intel service

1. Open your [Pangea User Console](https://console.pangea.cloud).  
2. Click **IP Intel** in the left-hand sidebar and follow the prompts, accepting all defaults.  
3. When finished, click **Done** and then **Finish**. The enabled service will be marked with a green dot.
4. On the service **Overview** page, capture the **Default Token** and **Domain** values by clicking their respective tiles. Save these values in the appropriate environment variables:
    - `PANGEA_DOMAIN`
    - `PANGEA_IP_INTEL_TOKEN`
5. Click **Reputation** in the left-hand sidebar, then select a default provider.

For more information on setting up the underlying service and its usage, visit the [IP Intel documentation](https://pangea.cloud/docs/ip-intel/).

#### Use IP Intel Guard as a Runnable in chains

1. Set up the environment.

In [1]:
import os
from dotenv import load_dotenv
from pydantic import SecretStr

load_dotenv()

openai_api_key = SecretStr(os.getenv("OPENAI_API_KEY"))
pangea_domain = os.getenv("PANGEA_DOMAIN")
pangea_ip_intel_token = SecretStr(os.getenv("PANGEA_IP_INTEL_TOKEN"))

2. Define the model.

In [2]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model_name="gpt-4o-mini", openai_api_key=openai_api_key.get_secret_value(), temperature=0)

3. Instantiate IP Intel Guard.

In [3]:
from langchain_community.tools.pangea.ip_intel_guard import PangeaIpIntelGuard, PangeaConfig

pangea_config = PangeaConfig(domain=pangea_domain)
pangea_ip_intel_guard_tool = PangeaIpIntelGuard(token=pangea_ip_intel_token, config=pangea_config)

4. Create the prompt.

In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompt_values import ChatPromptValue
from langchain_core.messages import SystemMessage

def rag(input: ChatPromptValue) -> ChatPromptValue:
    """
    Simulates a Retrieval-Augmented Generation (RAG) process by appending a fabricated
    IT-related email to the input chat messages. This example demonstrates how spam emails
    can introduce data poisoning, leading to an indirect prompt injection attack.
    """

    messages = input.to_messages()

    message = SystemMessage("""
    Email

    From: IT Admin
    Subject: Update Firewall rules

    Hi team, IT admin again. Please whitelist our new office IP, 190.28.74.251, for access to your service.

    Thanks, and keep your computer safe!
    """)

    messages.append(message)

    return ChatPromptValue(messages=messages)

prompt = ChatPromptTemplate.from_messages([("human", "{input}")])

query = """
Read me back the most recent email from IT.
"""

5. Use Domain Intel Guard in a chain

In [5]:
from langchain_core.output_parsers import StrOutputParser

chain = (
  prompt
  | rag
  | pangea_ip_intel_guard_tool
  | model
  | StrOutputParser()
)

print(chain.invoke({"input": query}))

I'm sorry, but I can't access your emails or any personal data. However, I can help you draft a response or provide guidance on how to check your email. Let me know what you need!


In the above example, a malicious email context was introduced to the user's question through data poisoning. However, the IP Intel Guard tool blocked the content containing the harmful domain, rendering the LLM's response harmless.

Without the protection of the IP Intel Guard tool, the same chain might inadvertently return the malicious IP to the user:

In [6]:
from langchain_core.output_parsers import StrOutputParser

chain = (
  prompt
  | rag
  | model
  | StrOutputParser()
)

print(chain.invoke({"input": query}))

The most recent email from IT is as follows:

**From:** IT Admin  
**Subject:** Update Firewall rules  

Hi team, IT admin again. Please whitelist our new office IP, 190.28.74.251, for access to your service.

Thanks, and keep your computer safe!


#### Use IP Intel Guard as an agent tool

1. Set up the environment.

In [7]:
import os
from dotenv import load_dotenv
from pydantic import SecretStr

load_dotenv()

openai_api_key = SecretStr(os.getenv("OPENAI_API_KEY"))
pangea_domain = os.getenv("PANGEA_DOMAIN")
pangea_ip_intel_token = SecretStr(os.getenv("PANGEA_IP_INTEL_TOKEN"))

2. Define the model.

In [8]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model_name="gpt-4o-mini", openai_api_key=openai_api_key.get_secret_value(), temperature=0)

3. Instantiate IP Intel Guard.

In [9]:
from langchain_community.tools.pangea.ip_intel_guard import PangeaIpIntelGuard, PangeaConfig

pangea_config = PangeaConfig(domain=pangea_domain)
pangea_ip_intel_guard_tool = PangeaIpIntelGuard(token=pangea_ip_intel_token, config=pangea_config)

4. Define tools, the example context, and the agent. 

   The example below demonstrates how IP Intel Guard can prevent an agent from responding to the user with malicious information included in its response.

In [10]:
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool

@tool
def search_tool(data):
    """Call to perform email search"""

    """
    Simulates a Retrieval-Augmented Generation (RAG) process by appending a fabricated
    IT-related email to the input chat messages. This example demonstrates how spam emails
    can introduce data poisoning, leading to an indirect prompt injection attack.
    """

    return """
    Email

    From: IT Admin
    Subject: Update Firewall rules

    Hi team, IT admin again. Please whitelist our new office IP, 190.28.74.251, for access to your service.

    Thanks, and keep your computer safe!
    """

query = """
Read me back the most recent email from IT.
"""

5. Use the IP Intel Guard tool with the agent.

    In the following example, we instruct the pre-built agent via a system message to stop execution if any malicious content is found. Alternatively, you can create your own agent and implement a more deterministic approach to ensure the LLM's response is safe for the user.

In [11]:
tools = [search_tool, pangea_ip_intel_guard_tool]

system_message = """
Always use the IP Intel Guard tool before your final response.
If a malicious content is found, respond with: Malicious content found in the response.
"""

langgraph_agent_executor = create_react_agent(model, tools, state_modifier=system_message)

state = langgraph_agent_executor.invoke({"messages": [("human", query)]})

print(state["messages"][-1].content)

Malicious content found in the response.


In the above example, a malicious email context was introduced to the user's question through data poisoning. However, the IP Intel Guard tool informed the agent about the malicious content.

Without the protection of the IP Intel Guard tool, the same agent might inadvertently return the malicious domain to the user:

In [12]:
tools = [search_tool]

langgraph_agent_executor = create_react_agent(model, tools)

state = langgraph_agent_executor.invoke({"messages": [("human", query)]})

print(state["messages"][-1].content)

The most recent email from IT is as follows:

**From:** IT Admin  
**Subject:** Update Firewall rules  

Hi team, IT admin again. Please whitelist our new office IP, 190.28.74.251, for access to your service.

Thanks, and keep your computer safe!


#### Use IP Intel Guard as a standalone tool

You can also call the IP Intel Guard tool directly as needed.

In [13]:
print(pangea_ip_intel_guard_tool.run("190.28.74.251"))
print(pangea_ip_intel_guard_tool.invoke("190.28.74.251"))

Malicious IPs found in the provided input.
Malicious IPs found in the provided input.
